# Song Finder - Example

This notebook demonstrates how to use the SongFinder class to search and retrieve songs from the database. The SongFinder provides:
- Direct song retrieval by ID
- Search by various criteria (author, year, genre, singer)
- Fuzzy name matching for song titles
- Finding dissimilar songs for ML training

First, let's import our required components and set up the finder with test configuration.

In [ ]:
from harinero.core.database import SongFinder
from harinero.utils import load_config
from pathlib import Path

# Load test configuration
config = load_config("/path/to/config.json") # Path to your config file

# Initialize SongFinder
finder = SongFinder(config)
print("SongFinder initialized with test configuration")

## 1. Basic Song Retrieval

Let's start by retrieving a song by its ID and examining its properties.

In [ ]:
# Get a song by ID
try:
    song = finder.get_song(1)
    print("Found song:")
    print(song)
except ValueError as e:
    print(f"Error: {e}")

# Try invalid ID
try:
    invalid_song = finder.get_song(-1)
except ValueError as e:
    print(f"\nExpected error with invalid ID: {e}")

## 2. Searching by Criteria

The SongFinder allows searching for songs using various criteria. All criteria are optional and can be combined.

In [ ]:
# Search by author
print("Songs by D'Arienzo:")
darienzo_songs = finder.get_songs_by_criteria(author_name="Juan D'Arienzo")
for song in darienzo_songs[:3]:  # Show first 3
    print(f"- {song.name} ({song.year})")

# Search by year range
print("\nSongs from 1940-1945:")
forties_songs = finder.get_songs_by_criteria(year=(1940, 1945))
for song in forties_songs[:3]:
    print(f"- {song.name} by {song.author_name} ({song.year})")

# Combined search
print("\nTangos by D'Arienzo from 1940-1945:")
specific_songs = finder.get_songs_by_criteria(
    author_name="Juan D'Arienzo",
    year=(1940, 1945),
    genre="tango"
)
for song in specific_songs[:3]:
    print(f"- {song.name} ({song.year})")

## 3. Fuzzy Name Matching

The SongFinder can find songs with similar names, handling spelling variations and accents.

In [ ]:
# Search with exact name
print("Searching for 'La Cumparsita':")
matches, details = finder.get_songs_by_name("La Cumparsita")
for song in matches:
    print(f"- {song.name} by {song.author_name} ({song.year})")

# Search with misspelling
print("\nSearching with misspelling 'La Comparsita':")
matches, details = finder.get_songs_by_name("La Comparsita")
for song in matches:
    print(f"- {song.name} by {song.author_name} ({song.year})")

# Search without accents
print("\nSearching without accent 'Punalada':")
matches, details = finder.get_songs_by_name("Punalada")
for song in matches:
    print(f"- {song.name} by {song.author_name} ({song.year})")

## 4. Finding Dissimilar Songs

For ML training, we often need to find songs that are intentionally dissimilar to a reference song.

In [ ]:
# Get a reference song
reference_song = finder.get_song(1)

# Find dissimilar songs
print(f"Finding songs dissimilar to: {reference_song.name}")
dissimilar_songs = finder.get_dissimilar_songs(
    reference_song,
    sample_size=5,  # Get 5 dissimilar songs
    year_range=10   # Weight by 10-year range
)

print("\nDissimilar songs found:")
for song in dissimilar_songs:
    print(f"- {song.name} by {song.author_name}")
    print(f"  Genre: {song.genre}, Year: {song.year}, Singer: {song.singer}")

## Handling Errors

Let's see how SongFinder handles various error conditions.

In [ ]:
# Try invalid author
try:
    songs = finder.get_songs_by_criteria(author_name="Nonexistent Orchestra")
except ValueError as e:
    print(f"Invalid author error: {e}")

# Try invalid genre
try:
    songs = finder.get_songs_by_criteria(genre="invalid_genre")
except ValueError as e:
    print(f"\nInvalid genre error: {e}")

# Try invalid year format
try:
    songs = finder.get_songs_by_criteria(year="1940")
except ValueError as e:
    print(f"\nInvalid year format error: {e}")

## Best Practices for Using SongFinder

1. **Error Handling**:
   - Always wrap SongFinder calls in try/except blocks
   - Handle NotFoundError for missing songs
   - Verify search criteria before querying

2. **Search Optimization**:
   - Use the most specific criteria available
   - Combine criteria to narrow results
   - Consider year ranges instead of exact years

3. **Name Matching**:
   - Use `get_songs_by_name()` for fuzzy matching
   - Normalize names before searching
   - Check multiple name variations if needed

4. **ML Dataset Creation**:
   - Use `get_dissimilar_songs()` for negative examples
   - Adjust year_range based on your needs
   - Verify sample_size is appropriate

5. **Performance**:
   - Cache frequently accessed songs
   - Batch similar queries together
   - Use specific criteria to reduce result sets